In [28]:
# Cell 1: Imports + DB connection


import sqlite3
import random
import datetime as dt

# Path to my database file
db_path = r"C:\Yasaswini Sure\Restaurant_Food_Ordering_System_UberEats.db"

# Connect to SQLite
conn = sqlite3.connect(db_path)
cur = conn.cursor()

# Enable FK support
cur.execute("PRAGMA foreign_keys = ON;")

# Fix random seed for consistent results
random.seed(24)

print("Connected to:", db_path)


Connected to: C:\Yasaswini Sure\Restaurant_Food_Ordering_System_UberEats.db


In [29]:
# Cell 2: Create all database tables

schema_sql = """
PRAGMA foreign_keys = ON;

CREATE TABLE IF NOT EXISTS restaurants (
    restaurant_id   INTEGER PRIMARY KEY,
    restaurant_name TEXT NOT NULL,
    cuisine_type    TEXT NOT NULL,
    city            TEXT NOT NULL,
    avg_rating      REAL CHECK (avg_rating BETWEEN 0 AND 5),
    is_active       INTEGER NOT NULL CHECK (is_active IN (0,1))
);

CREATE TABLE IF NOT EXISTS customers (
    customer_id       INTEGER PRIMARY KEY,
    first_name        TEXT NOT NULL,
    last_name         TEXT NOT NULL,
    email             TEXT NOT NULL UNIQUE,
    phone_number      TEXT,
    city              TEXT NOT NULL,
    postcode          TEXT NOT NULL,
    loyalty_tier      INTEGER NOT NULL CHECK (loyalty_tier BETWEEN 1 AND 3),
    registration_date TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS delivery_partners (
    partner_id     INTEGER PRIMARY KEY,
    partner_name   TEXT NOT NULL,
    vehicle_type   TEXT NOT NULL,
    partner_rating INTEGER NOT NULL CHECK (partner_rating BETWEEN 1 AND 5),
    is_active      INTEGER NOT NULL CHECK (is_active IN (0,1))
);

CREATE TABLE IF NOT EXISTS menu_items (
    item_id       INTEGER PRIMARY KEY,
    restaurant_id INTEGER NOT NULL,
    item_name     TEXT NOT NULL,
    category      TEXT NOT NULL,
    unit_price    REAL NOT NULL CHECK (unit_price > 0),
    is_available  INTEGER NOT NULL CHECK (is_available IN (0,1)),
    FOREIGN KEY (restaurant_id) REFERENCES restaurants(restaurant_id)
);

CREATE TABLE IF NOT EXISTS orders (
    order_id         INTEGER PRIMARY KEY,
    customer_id      INTEGER NOT NULL,
    restaurant_id    INTEGER NOT NULL,
    partner_id       INTEGER,
    order_datetime   TEXT NOT NULL,
    delivery_minutes INTEGER,
    order_status     TEXT NOT NULL,
    subtotal_amount  REAL NOT NULL,
    discount_amount  REAL NOT NULL,
    delivery_fee     REAL NOT NULL,
    total_amount     REAL NOT NULL,
    payment_status   TEXT NOT NULL,
    FOREIGN KEY(customer_id)   REFERENCES customers(customer_id),
    FOREIGN KEY(restaurant_id) REFERENCES restaurants(restaurant_id),
    FOREIGN KEY(partner_id)    REFERENCES delivery_partners(partner_id)
);

CREATE TABLE IF NOT EXISTS order_items (
    order_id        INTEGER NOT NULL,
    item_id         INTEGER NOT NULL,
    quantity        INTEGER NOT NULL CHECK(quantity > 0),
    unit_price_used REAL NOT NULL,
    PRIMARY KEY(order_id, item_id),
    FOREIGN KEY(order_id) REFERENCES orders(order_id),
    FOREIGN KEY(item_id)  REFERENCES menu_items(item_id)
);

CREATE TABLE IF NOT EXISTS payments (
    payment_id      INTEGER PRIMARY KEY,
    order_id        INTEGER NOT NULL,
    payment_method  TEXT NOT NULL,
    payment_datetime TEXT NOT NULL,
    amount_paid     REAL NOT NULL CHECK(amount_paid >= 0),
    FOREIGN KEY(order_id) REFERENCES orders(order_id)
);
"""

cur.executescript(schema_sql)
conn.commit()

print("Schema created successfully.")


Schema created successfully.


In [30]:
# Cell 3: Clear old rows (clean restart)


cur.execute("DELETE FROM payments;")
cur.execute("DELETE FROM order_items;")
cur.execute("DELETE FROM orders;")
cur.execute("DELETE FROM menu_items;")
cur.execute("DELETE FROM delivery_partners;")
cur.execute("DELETE FROM customers;")
cur.execute("DELETE FROM restaurants;")
conn.commit()

print("Old data cleared.")


Old data cleared.


In [31]:
# Cell 4: Helper lists + simple functions


cities = ["London", "Manchester", "Birmingham", "Leeds", "Glasgow", "Liverpool"]
cuisines = ["Indian", "Italian", "Chinese", "Thai", "American", "Mexican"]

first_names = ["Rahul","Priya","Emily","James","Aarav","Sophia","Liam","Olivia","Amir","Sara","David","Chloe","Karan","Meera","Jack","Mia"]
last_names  = ["Sharma","Patel","Khan","Brown","Taylor","Singh","Wilson","Clark","Reddy","Gupta","Evans","Hall","Wright","Walker","Lewis","King"]

email_domains = ["gmail.com","mail.com","example.com","outlook.com"]
vehicle_types = ["Bike","Scooter","Car","Cycle"]

postcode_prefixes = ["EC1","EC2","N1","SE1","SW1","E14","W1","NW1","M1","B1"]

# Menu templates for each cuisine
menu_templates = {
    "Indian":   ["Chicken Biryani","Paneer Masala","Dal Tadka","Butter Chicken","Masala Dosa"],
    "Italian":  ["Margherita Pizza","Pasta Alfredo","Lasagna","Garlic Bread","Bruschetta"],
    "Chinese":  ["Veg Noodles","Fried Rice","Manchurian","Spring Rolls"],
    "Thai":     ["Pad Thai","Green Curry","Red Curry","Tom Yum Soup"],
    "American": ["Cheeseburger","Fries","Chicken Wings","BBQ Ribs"],
    "Mexican":  ["Tacos","Burrito","Quesadilla","Nachos"]
}

categories = ["Starter","Main","Dessert","Beverage","Side"]

def random_phone():
    return "+44 7" + "".join(str(random.randint(0,9)) for _ in range(9))

def random_postcode():
    return f"{random.choice(postcode_prefixes)} {random.randint(1,9)}{random.choice(['AA','AB','BA','BB'])}"

def random_date(start=2023, end=2025):
    start_d = dt.date(start,1,1)
    end_d   = dt.date(end,11,1)
    diff = (end_d - start_d).days
    pick = start_d + dt.timedelta(days=random.randint(0, diff))
    return pick.isoformat()

def random_datetime(start=2023, end=2025):
    start_dt = dt.datetime(start,1,1,0,0)
    end_dt   = dt.datetime(end,11,1,23,59)
    total = int((end_dt - start_dt).total_seconds())
    offset = random.randint(0, total)
    return (start_dt + dt.timedelta(seconds=offset)).strftime("%Y-%m-%d %H:%M:%S")


In [32]:
# Cell 5: Insert restaurant data


restaurant_seed = [
    ("Spice Junction","Indian","London"),
    ("Curry House","Indian","Manchester"),
    ("Pizza Hub","Italian","London"),
    ("Pasta Street","Italian","Birmingham"),
    ("Dragon Wok","Chinese","Leeds"),
    ("Beijing Express","Chinese","London"),
    ("Bangkok Bites","Thai","Manchester"),
    ("Thai Corner","Thai","Glasgow"),
    ("Burger Station","American","Liverpool"),
    ("Grill & Fries","American","London"),
    ("Taco Fiesta","Mexican","Leeds"),
    ("Burrito Bar","Mexican","Birmingham")
]

rows = []
for i,(name,cuisine,city) in enumerate(restaurant_seed, start=1):
    rows.append((i,name,cuisine,city,round(random.uniform(3.5,5.0),1),1))

cur.executemany("""
    INSERT INTO restaurants VALUES (?, ?, ?, ?, ?, ?);
""", rows)

conn.commit()
print("Restaurants inserted:", len(rows))


Restaurants inserted: 12


In [33]:
# Cell 6: Generate delivery riders


for pid in range(1, 81):
    fn = random.choice(first_names)
    ln = random.choice(last_names)
    cur.execute("""
        INSERT INTO delivery_partners (
            partner_id, partner_name, vehicle_type,
            partner_rating, is_active
        ) VALUES (?, ?, ?, ?, ?);
    """, (
        pid, f"{fn} {ln}", random.choice(vehicle_types),
        random.randint(3,5), 1
    ))

conn.commit()
print("Delivery partners inserted:", 80)


Delivery partners inserted: 80


In [34]:
# Cell 7: Insert customer records


for cid in range(1, 601):
    fn = random.choice(first_names)
    ln = random.choice(last_names)
    cur.execute("""
        INSERT INTO customers (
            customer_id, first_name, last_name,
            email, phone_number, city, postcode,
            loyalty_tier, registration_date
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
    """, (
        cid, fn, ln,
        f"{fn.lower()}.{ln.lower()}{random.randint(1,999)}@{random.choice(email_domains)}",
        random_phone(),
        random.choice(cities),
        random_postcode(),
        random.randint(1,3),
        random_date()
    ))

conn.commit()
print("Customers inserted:", 600)


Customers inserted: 600


In [35]:
# Cell 8: Insert menu items


cur.execute("SELECT restaurant_id, cuisine_type FROM restaurants;")
rest_list = cur.fetchall()

item_id = 1
for rid, cuisine in rest_list:
    template = menu_templates.get(cuisine, ["Chef Special"])
    n = random.randint(15, 30)

    for _ in range(n):
        base = random.choice(template)
        category = random.choice(categories)

        if category == "Main":
            price = random.uniform(7, 16)
        elif category == "Starter":
            price = random.uniform(3, 7)
        elif category == "Dessert":
            price = random.uniform(3, 7)
        elif category == "Beverage":
            price = random.uniform(1.5, 5)
        else:
            price = random.uniform(2, 6)

        cur.execute("""
            INSERT INTO menu_items VALUES (?, ?, ?, ?, ?, ?);
        """, (
            item_id, rid, base, category, round(price,2),
            1 if random.random() < 0.95 else 0
        ))

        item_id += 1

conn.commit()
print("Menu items inserted.")


Menu items inserted.


In [36]:
# Cell 9: Insert orders, order items, and payment records


# Cache restaurant and partner lists
cur.execute("SELECT restaurant_id FROM restaurants;")
restaurant_ids = [row[0] for row in cur.fetchall()]

cur.execute("SELECT partner_id FROM delivery_partners WHERE is_active=1;")
active_partners = [row[0] for row in cur.fetchall()]

# Cache menu items per restaurant
menu_cache = {}
for rid in restaurant_ids:
    cur.execute("SELECT item_id, unit_price FROM menu_items WHERE restaurant_id=? AND is_available=1;", (rid,))
    menu_cache[rid] = cur.fetchall()

payment_id = 1

for oid in range(1, 1201):
    customer_id = random.randint(1, 600)
    rest_id = random.choice(restaurant_ids)
    menu_list = menu_cache[rest_id]

    if not menu_list:
        continue

    # Delivery or pickup
    if random.random() < 0.8:
        partner_id = random.choice(active_partners)
        delivery_min = random.randint(20, 60)
        delivery_fee = round(random.uniform(1.5, 5.0), 2)
    else:
        partner_id = None
        delivery_min = None
        delivery_fee = 0.0

    order_ts = random_datetime()

    # Build order items first
    n_items = random.randint(1, 5)
    selected = random.sample(menu_list, k=min(n_items, len(menu_list)))

    subtotal = 0
    order_item_rows = []
    for item, price in selected:
        qty = random.randint(1, 4)
        subtotal += qty * price
        order_item_rows.append((item, price, qty))

    subtotal = round(subtotal, 2)

    # Discount
    if random.random() < 0.25:
        discount = round(subtotal * random.uniform(0.05, 0.25), 2)
    else:
        discount = 0.0

    total = round(subtotal - discount + delivery_fee, 2)

    # Status flags
    if random.random() < 0.8:
        order_status = "Delivered"
        pay_status = "Paid"
    else:
        order_status = random.choice(["Placed","Preparing","On the way","Cancelled"])
        pay_status = "Refunded" if order_status == "Cancelled" else random.choice(["Pending","Failed","Paid"])

    # Insert order row first
    cur.execute("""
        INSERT INTO orders VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    """, (
        oid, customer_id, rest_id, partner_id,
        order_ts, delivery_min, order_status,
        subtotal, discount, delivery_fee, total, pay_status
    ))

    # Insert order items
    for item, price, qty in order_item_rows:
        cur.execute("""
            INSERT INTO order_items VALUES (?, ?, ?, ?);
        """, (oid, item, qty, round(price,2)))

    # Insert payment if applicable
    if pay_status in ("Paid","Refunded"):
        method = random.choice(["Card","UPI","Wallet","Cash"])
        cur.execute("""
            INSERT INTO payments VALUES (?, ?, ?, ?, ?);
        """, (payment_id, oid, method, order_ts, total))
        payment_id += 1

conn.commit()
print("Orders + items + payments inserted.")


Orders + items + payments inserted.


In [37]:
# Cell 10: Final table counts


tables = ["restaurants","customers","delivery_partners","menu_items","orders","order_items","payments"]

print("\nFinal row counts:")
for t in tables:
    cur.execute(f"SELECT COUNT(*) FROM {t};")
    print(f"{t:22s}", cur.fetchone()[0])



Final row counts:
restaurants            12
customers              600
delivery_partners      80
menu_items             267
orders                 1200
order_items            3607
payments               1095


In [38]:
# Cell 11: Close database

conn.close()
print("Connection closed.")


Connection closed.
